<a href="https://colab.research.google.com/github/vanshul27/EDA-and-Prediction-on-Wine-Reviews/blob/main/Model_Wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


**Importing Original Datset and the Dataset without Outliers**

In [5]:
df = pd.read_csv("/content/drive/MyDrive/I_Task/OSX_DS_assignment.csv")
dfm = pd.read_csv("/content/drive/MyDrive/I_Task/OSX_DS_assignment_MODI.csv")

**We will be classifying the variety of wines using Reviews**

In [6]:
df.head()

,user_name,country,review_title,review_description,designation,points,price,province,region_1,region_2,winery,variety
0,@kerinokeefe,Italy,Nicosia 2013 Vulkà Bianco (Etna),"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Nicosia,White Blend
1,@vossroger,Portugal,Quinta dos Avidagos 2011 Avidagos Red (Douro),"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Quinta dos Avidagos,Portuguese Red
2,@paulgwine,US,Rainstorm 2013 Pinot Gris (Willamette Valley),"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Rainstorm,Pinot Gris
3,NaN,US,St. Julian 2013 Reserve Late Harvest Riesling ...,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,St. Julian,Riesling
4,@paulgwine,US,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks,Pinot Noir


In [23]:
##Using Wine Review and Price as a feature of our model
X = df.drop(['user_name','country','review_title','designation','points','province','region_1','region_2','winery','variety'], axis = 1)
y = df.variety

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(77491, 2) (25831, 2) (77491,) (25831,)


In [63]:
X_train.dropna()
X_test.dropna()
y_train.dropna()
y_test.dropna()

26328                  White Blend
91304                   Pinot Noir
65110                   Chardonnay
59683     Bordeaux-style Red Blend
41357                       Malbec
                    ...           
103312                  Pinot Gris
99134                     Riesling
82629                   Pinot Noir
50433                  White Blend
84385                       Merlot
Name: variety, Length: 25831, dtype: object

ValueError: ignored

In [43]:
wine =df.variety.unique().tolist()
wine.sort()
wine[:10]

['Bordeaux-style Red Blend',
 'Bordeaux-style White Blend',
 'Cabernet Franc',
 'Cabernet Sauvignon',
 'Champagne Blend',
 'Chardonnay',
 'Gamay',
 'Gewürztraminer',
 'Grüner Veltliner',
 'Malbec']

In [31]:
'''output = set()
for x in df.variety:
    x = x.lower()
    x = x.split()
    for y in x:
        output.add(y)

variety_list =sorted(output)
variety_list[:10]
'''

'output = set()\nfor x in df.variety:\n    x = x.lower()\n    x = x.split()\n    for y in x:\n        output.add(y)\n\nvariety_list =sorted(output)\nvariety_list[:10]\n'

In [29]:
'''extras = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', 'cab',"%"]
import nltk
nltk.download('stopwords')
stop = set(stopwords.words('english'))
stop.update(variety_list)
stop.update(extras)'''

'extras = [\'.\', \',\', \'"\', "\'", \'?\', \'!\', \':\', \';\', \'(\', \')\', \'[\', \']\', \'{\', \'}\', \'cab\',"%"]\nimport nltk\nnltk.download(\'stopwords\')\nstop = set(stopwords.words(\'english\'))\nstop.update(variety_list)\nstop.update(extras)'

In [51]:
from scipy.sparse import hstack

vect = CountVectorizer()
X_train_dtm = vect.fit_transform(X_train.review_description)
price = X_train.price.values[:,None]
X_train_dtm = hstack((X_train_dtm, price))
X_train_dtm

<77491x25368 sparse matrix of type '<class 'numpy.float64'>'
	with 2754494 stored elements in COOrdinate format>

In [57]:
X_test_dtm = vect.transform(X_test.review_description)
price_test = X_test.price.values[:,None]
X_test_dtm = hstack((X_test_dtm, price_test))

In [64]:
from sklearn.linear_model import LogisticRegression
models = {}
for z in wine:
    model = LogisticRegression()
    y = y_train == z
    model.fit(X_train_dtm, y_train)
    models[z] = model

testing_probs = pd.DataFrame(columns = wine)

ValueError: ignored

In [54]:
for variety in wine:
    testing_probs[variety] = models[variety].predict_proba(X_test_dtm)[:,1]
    
predicted_wine = testing_probs.idxmax(axis=1)

comparison = pd.DataFrame({'actual':y_test.values, 'predicted':predicted_wine.values})   

from sklearn.metrics import accuracy_score
print('Accuracy Score:',accuracy_score(comparison.actual, comparison.predicted)*100,"%")
comparison.head(5)

KeyError: ignored